In [1]:
# Core libraries
import numpy as np
import pandas as pd

# System 
from pathlib import Path
import subprocess
from shutil import copy
import datetime

# Simulation
import tellurium as te
import roadrunner

In [2]:
%load_ext watermark

In [3]:
%watermark --iversions

tellurium 2.1.3
numpy     1.14.3
roadrunner1.4.24
pandas    0.22.0



### Setup paths and helper functions

In [4]:
models_path = Path("../data/models")
simulation_path = Path("../data/simulation_results")

In [5]:
def run_matlab_script(script_path):
    # script_path - Path object to the m script
    str_script = str((script_path).resolve())
    run_command = f""" -r "run('{str_script}');exit;" """

    matlab_run = subprocess.run(
        ["matlab", "-nodisplay", "-nosplash", "-nodesktop", run_command],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )

    # check for errors
    if matlab_run.returncode == 0:
        print("Run ended succesfully")
    else:
        print("Run ended with an error")

    return matlab_run.stdout.decode("UTF-8")

## Problem setup
There are three strains - WT, zwf knockout, zwf overexpressed. Their growth rates are 0.65, 0.60, 0.66 accordingly in batch fermentation.

# Simulations of Chassagnole model

In [6]:
model = te.loadAntimonyModel(
    te.sbmlToAntimony(str(models_path / "modified_files" / "Chassagnole2002_modified.xml"))
)

### Run the simulations
Simulations are being performed as ODEs integration for 1e5 time units. Results after the end of simulation are being used as "steady-state" values.

In [7]:
# WT simulation
model.resetToOrigin()

sample_id = "WT"
print(f"Working on sample {sample_id}")

# For Dilution rate approx. equal to 0.2 h-1
model.simulate(0, 10000)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})

save_path = simulation_path / "Chassagnole" / "zwf_pgi_eno_sensitivity"
df.to_csv(save_path / f"chassagnole_sens_WT.csv")

Working on sample WT


## Perform ZWF sensitivity analysis

In [9]:
# simulate ZWF KO
# set the original state
model.resetToOrigin()
# For Dilution rate approx. equal to 0.2 h-1
    
sample_id = "dzwf"
print(f"Working on sample {sample_id}")
# perform poor-mans 'knockout'
model.setValue("vG6PDH_rmaxG6PDH", 0)

# Instead of steady-state solver use long integration.
# make the "fair" comparison because all other models run not to steady state
model.simulate(0, 10000)
 
ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Chassagnole" / "zwf_pgi_eno_sensitivity" / f"chassagnole_{sample_id}.csv")

Working on sample dzwf


In [10]:
# simulate ZWF overexpression
# set the original state
model.resetToOrigin()
# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4
    
sample_id = "zwf(15)"
print(f"Working on sample {sample_id}")
# perform overexpression
vmax = model.vG6PDH_rmaxG6PDH
model.setValue("vG6PDH_rmaxG6PDH", vmax * 15)

# Instead of steady-state solver use long integration.
# make the "fair" comparison because all other models run not to steady state
model.simulate(0, 10000)
 
ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Chassagnole" / "zwf_pgi_eno_sensitivity" / f"chassagnole_{sample_id}.csv")

Working on sample zwf(15)


## Perform PGI sensitivity analysis

In [11]:
# simulate PGI KO
# set the original state
model.resetToOrigin()
# For Dilution rate approx. equal to 0.2 h-1
    
sample_id = "dpgi"
print(f"Working on sample {sample_id}")
# perform poor-mans 'knockout'
model.setValue("vPGI_rmaxPGI", 0)

# Instead of steady-state solver use long integration.
# make the "fair" comparison because all other models run not to steady state
model.simulate(0, 10000)
 
ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Chassagnole" / "zwf_pgi_eno_sensitivity" / f"chassagnole_{sample_id}.csv")

Working on sample dpgi


In [12]:
# simulate various pgi levels
experiments = [
    {"sample_id": "pgi(0)", "level": 0.2},
    {"sample_id": "pgi(20)", "level": 1.2},
    {"sample_id": "pgi(50)", "level": 2.4},
    {"sample_id": "pgi(100)", "level": 4.1},
]
for exp in experiments:
    # set the original state
    model.resetToOrigin()
    # For Dilution rate approx. equal to 0.2 h-1

    sample_id = exp["sample_id"]
    print(f"Working on sample {sample_id}")
    # perform overexpression
    vmax = model.vPGI_rmaxPGI
    model.setValue("vPGI_rmaxPGI", vmax * exp["level"])

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    model.simulate(0, 10000)

    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()

    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv( simulation_path / "Chassagnole" / "zwf_pgi_eno_sensitivity" / f"chassagnole_{sample_id}.csv")

Working on sample pgi(0)
Working on sample pgi(20)
Working on sample pgi(50)
Working on sample pgi(100)


## Perform ENO sensitivity analysis

In [13]:
# simulate various eno levels
experiments = [
    {"sample_id": "eno(0)", "level": 0.2},
    {"sample_id": "eno(50)", "level": 1.8},
    {"sample_id": "eno(200)", "level": 3.0},
    {"sample_id": "eno(500)", "level": 3.1},
]
for exp in experiments:
    # set the original state
    model.resetToOrigin()

    sample_id = exp["sample_id"]
    print(f"Working on sample {sample_id}")
    # perform overexpression
    vmax = model.vENO_rmaxENO
    model.setValue("vENO_rmaxENO", vmax * exp["level"])

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    model.simulate(0, 10000)

    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()

    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv( simulation_path / "Chassagnole" / "zwf_pgi_eno_sensitivity" / f"chassagnole_{sample_id}.csv")

Working on sample eno(0)
Working on sample eno(50)
Working on sample eno(200)
Working on sample eno(500)


# Simulations of Millard model

### Define the model and experimental setup

In [54]:
# load as Antimony bypasses the lack of ability to change local parameters in native SBML model
model = te.loadAntimonyModel(
    te.sbmlToAntimony(
        str(models_path / "original_files" / "Millard_2017_MODEL1505110000_url.xml")
    )
)

### Run the simulations
Simulations are being performed as ODEs integration for 1e5 time units. Results after the end of simulation are being used as "steady-state" values.

In [55]:
# WT simulation
model.resetToOrigin()

sample_id = "WT"
print(f"Working on sample {sample_id}")

# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4
model.simulate(0, 10000)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})

save_paths = ["zwf", "pgi", "eno"]
for path in save_paths:
    save_path = simulation_path / "Millard" / f"{path}_sensitivity"
    save_path.mkdir(parents=True, exist_ok=True)
    df.to_csv(save_path / f"millard_{path}_sens_WT.csv")

Working on sample WT


## Perform ZWF sensitivity analysis

In [46]:
# simulate ZWF KO
# set the original state
model.resetToOrigin()
# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4
    
sample_id = "dzwf"
print(f"Working on sample {sample_id}")
# perform poor-mans 'knockout'
model.setValue("ZWF_Vmax", 0)

# Instead of steady-state solver use long integration.
# make the "fair" comparison because all other models run not to steady state
model.simulate(0, 10000)
 
ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Millard" / "zwf_sensitivity" / f"millard_zwf_sens_{sample_id}.csv")

Working on sample dzwf


In [47]:
# simulate ZWF overexpression
# set the original state
model.resetToOrigin()
# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4
    
sample_id = "zwf(15)"
print(f"Working on sample {sample_id}")
# perform overexpression
vmax = model.ZWF_Vmax
model.setValue("ZWF_Vmax", vmax * 15)

# Instead of steady-state solver use long integration.
# make the "fair" comparison because all other models run not to steady state
model.simulate(0, 10000)
 
ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Millard" / "zwf_sensitivity" / f"millard_zwf_sens_{sample_id}.csv")

Working on sample zwf(15)


## Perform PGI sensitivity analysis

In [56]:
# simulate PGI KO
# set the original state
model.resetToOrigin()
# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4
    
sample_id = "dpgi"
print(f"Working on sample {sample_id}")
# perform poor-mans 'knockout'
model.setValue("PGI_Vmax", 0)

# Instead of steady-state solver use long integration.
# make the "fair" comparison because all other models run not to steady state
model.simulate(0, 10000)
 
ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Millard" / "pgi_sensitivity" / f"millard_pgi_sens_{sample_id}.csv")

Working on sample dpgi


In [57]:
# simulate various pgi levels
experiments = [
    {"sample_id": "pgi(0)", "level": 0.2},
    {"sample_id": "pgi(20)", "level": 1.2},
    {"sample_id": "pgi(50)", "level": 2.4},
    {"sample_id": "pgi(100)", "level": 4.1},
]
for exp in experiments:
    # set the original state
    model.resetToOrigin()
    # For Dilution rate approx. equal to 0.2 h-1
    model.FEED = 0.4

    sample_id = exp["sample_id"]
    print(f"Working on sample {sample_id}")
    # perform overexpression
    vmax = model.PGI_Vmax
    model.setValue("PGI_Vmax", vmax * exp["level"])

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    model.simulate(0, 10000)

    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()

    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv(
        simulation_path
        / "Millard"
        / "pgi_sensitivity"
        / f"millard_pgi_sens_{sample_id}.csv"
    )

Working on sample pgi(0)
Working on sample pgi(20)
Working on sample pgi(50)
Working on sample pgi(100)


## Perform ENO sensitivity analysis

In [58]:
# simulate various eno levels
experiments = [
    {"sample_id": "eno(0)", "level": 0.2},
    {"sample_id": "eno(50)", "level": 1.8},
    {"sample_id": "eno(200)", "level": 3.0},
    {"sample_id": "eno(500)", "level": 3.1},
]
for exp in experiments:
    # set the original state
    model.resetToOrigin()
    # For Dilution rate approx. equal to 0.2 h-1
    model.FEED = 0.4

    sample_id = exp["sample_id"]
    print(f"Working on sample {sample_id}")
    # perform overexpression
    vmax = model.ENO_Vmax
    model.setValue("ENO_Vmax", vmax * exp["level"])

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    model.simulate(0, 10000)

    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()

    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv(
        simulation_path
        / "Millard"
        / "eno_sensitivity"
        / f"millard_eno_sens_{sample_id}.csv"
    )

Working on sample eno(0)
Working on sample eno(50)
Working on sample eno(200)
Working on sample eno(500)


# Simulations of Khodayari model
It's MATLAB based model, so the simulation is being performed by invoking matlab in CLI mode. Simulation time is uncomfortably long

In [30]:
khod_path = models_path / "modified_files" / "Khodayari"

## Perform ZWF sensitivity analysis

In [28]:
zwf_script = khod_path / "Run_Khodayari_zwf_sensitivity.m"
result = run_matlab_script(script_path=zwf_script)

Run ended succesfully


Move simulation results to `simulation_results`

In [31]:
destination = simulation_path / "Khodayari" / f"zwf_sensitivity"
destination.mkdir(parents=True, exist_ok=True)
for file in khod_path.glob("khodayari_zwf_sens*.mat"):
    copy(file, destination)

## Perform PGI sensitivity analysis

In [32]:
pgi_script = khod_path / "Run_Khodayari_pgi_sensitivity.m"
result = run_matlab_script(script_path=pgi_script)

Run ended succesfully


Move simulation results to `simulation_results`

In [33]:
destination = simulation_path / "Khodayari" / f"pgi_sensitivity"
destination.mkdir(parents=True, exist_ok=True)
for file in khod_path.glob("khodayari_pgi_sens*.mat"):
    copy(file, destination)

## Perform ENO sensitivity analysis

In [34]:
eno_script = khod_path / "Run_Khodayari_eno_sensitivity.m"
result = run_matlab_script(script_path=eno_script)

Run ended succesfully


Move simulation results to `simulation_results`

In [36]:
destination = simulation_path / "Khodayari" / f"eno_sensitivity"
destination.mkdir(parents=True, exist_ok=True)
for file in khod_path.glob("khodayari_eno_sens*.mat"):
    copy(file, destination)

# Simulations of Kurata model
It's MATLAB based model, so the simulation is being performed by invoking matlab in CLI mode.

In [37]:
kurata_path = models_path / "modified_files" / "Kurata_continuous"

## Perform ZWF sensitivity analysis

In [38]:
zwf_script = kurata_path / "Run_Kurata_zwf_sensitivity.m"
result = run_matlab_script(script_path=zwf_script)

Run ended succesfully


Move simulation results to `simulation_results`

In [40]:
destination = simulation_path / "Kurata" / f"zwf_sensitivity"
destination.mkdir(parents=True, exist_ok=True)
for file in kurata_path.glob("kurata_zwf_sens*.mat"):
    copy(file, destination)

## Perform PGI sensitivity analysis

In [41]:
pgi_script = kurata_path / "Run_Kurata_pgi_sensitivity.m"
result = run_matlab_script(script_path=pgi_script)

Run ended succesfully


Move simulation results to `simulation_results`

In [43]:
destination = simulation_path / "Kurata" / f"pgi_sensitivity"
destination.mkdir(parents=True, exist_ok=True)
for file in kurata_path.glob("kurata_pgi_sens*.mat"):
    copy(file, destination)

## Perform ENO sensitivity analysis

In [44]:
eno_script = kurata_path / "Run_Kurata_eno_sensitivity.m"
result = run_matlab_script(script_path=eno_script)

Run ended succesfully


'\n                            < M A T L A B (R) >\n                  Copyright 1984-2017 The MathWorks, Inc.\n                   R2017b (9.3.0.713579) 64-bit (maci64)\n                             September 14, 2017\n\n \nTo get started, type one of these: helpwin, helpdesk, or demo.\nFor product information, visit www.mathworks.com.\n \nWorking on WT\nElapsed time is 9.733874 seconds.\nWorking on eno(0)\nElapsed time is 17.984527 seconds.\nWorking on eno(50)\nElapsed time is 23.773072 seconds.\nWorking on eno(200)\nElapsed time is 29.887313 seconds.\nWorking on eno(500)\nElapsed time is 34.613805 seconds.\n'

Move simulation results to `simulation_results`

In [45]:
destination = simulation_path / "Kurata" / f"eno_sensitivity"
destination.mkdir(parents=True, exist_ok=True)
for file in kurata_path.glob("kurata_eno_sens*.mat"):
    copy(file, destination)